# Trabalho 2

## 4.1

### Imports

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeRegressor, export_text

### Variáveis

In [ ]:
ficheiro = "../../ficheiros/Trabalho 2/Dados_Trabalho_TP2.csv"
ficheiro_dummies = "../../ficheiros/Trabalho 2/Dados_Trabalho_dummies_TP2.csv"
colunas_classes_binarias = ['Genero', 'Historico_obesidade_familiar', 'FCCAC', 'Fumador',
                            'MCC']  # Colunas de valores binários, ver 4.1.1.
colunas_numericas = ["Idade", "FCV", "NRP", "CA", "FAF", "TUDE", "IMC"]
colunas_classes = ['Genero', 'Historico_obesidade_familiar', 'FCCAC', 'Fumador', 'MCC']
colunas_classes_multiplos = ["CCER", "CBA", "TRANS"]

%matplotlib inline

skip_graficos = True

# Funções auxiliares

In [ ]:
def titulo(texto: str):
    print(f"\033[21;30;44m{texto}\033[0m")


def etiqueta_e_valor(etiqueta: str, valor: str = ""):
    print(f"\033[0;94m{etiqueta}: \033[1;94m{valor}\033[0m")


def texto(texto: str, cor="94"):
    print(f"\033[0;{cor}m{texto}\033[0m")


def undersplit(texto):
    return " ".join(texto.split("_"))


def histograma(
        dados: pd.Series,
        grupos: int = 20,
        kde: bool = True,
        size_x: int = 4,
        size_y: int = 4,
        titulo: str = "",
        etiqueta_x: str = "",
        etiqueta_y: str = "",
        color: str = "skyblue",
        edgecolor: str = "black",
):
    """Gera um histograma de forma isolada.
    
    A dimensão final do gráfico é controlada pela relação entre os valores size_x e size_y.
    """
    hist_fig, hist_axes = plt.subplots(figsize=(size_x, size_y))
    sns.histplot(dados, bins=grupos, kde=kde, color=color, edgecolor=edgecolor)
    hist_axes.set_title(titulo)
    hist_axes.set_xlabel(etiqueta_x)
    hist_axes.set_ylabel(etiqueta_y)
    plt.tight_layout()


def tarte(
        dados: pd.Series,
        size_x: int = 4,
        size_y: int = 4,
        autopct="%.2f%%",
        titulo: str = "",
        **kwargs,
):
    pie_fig, pie_axes = plt.subplots(figsize=(size_x, size_y))
    pie_axes.pie(
        dados,
        autopct=autopct,
        **kwargs
    )
    pie_fig.suptitle(titulo)
    return pie_fig, pie_axes


### Inicializações

In [ ]:
warnings.filterwarnings("ignore")  # Desabilitar warnbings.
plt.style.use(
    "style/estilo.mplstyle")  # Garantir que se utiliza um estilo definido centralmente e comum a todos os gráficos.
label_encoder = LabelEncoder()

### 4.1.1

#### Leitura de dados

Como os dados lidos não são em grande quantidade, vamos criar uma cópia do **dataset** para trabalharmos nesta, para poder ter acesso aos "originais" sem ter de os ler novamente do ficheiro.
Por exemplo em alguns dos gráficos, a utilização das *classes* (valores originais) como *etiqueta* é mais elucidativa que usar os valores codificados.  

Na análise ao **dataset** verifica-se que a primeira coluna, identificada como "Unnamed" tem o indíce de cada registo de dados, algo que nos é fornecido pelo **pandas**, e portanto podemos eliminar esta coluna. 
Esta operação estaria incluída nas tarefas do ponto *4.1.4*, mas realizar neste ponto facilita a criação dos gráficos no próximo ponto.

In [ ]:
dados_lidos = pd.read_csv(ficheiro)
dados_analise = dados_lidos.copy()
dados_analise.drop(columns=dados_lidos.columns[0], axis=1, inplace=True)

### Dimensão dos dados (lidos)

Os dados lidos incluem 1 coluna com o valor dos indices das linhas, que é removida de imediato.

In [ ]:
nr_linhas, nr_colunas = dados_lidos.shape
titulo("Dimensão dos dados")
etiqueta_e_valor("Número de linhas:", nr_linhas)
etiqueta_e_valor("Número de colunas:", nr_colunas)

### Análise do dataset

#### Atributos

Descrição dos atributos, segundo a documentação. 

| Sigla | Descrição                                                         |
|-------|-------------------------------------------------------------------|
| FCCAC | Frequência de Consumo de Comida Altamente Calórica                | 
| FCV   | Frequência de Consumo de Vegetais                                 | 
| NRP   | Número de Refeições Principais                                    |
| CCER  | Consumo de Comida Entre Refeições                                 |
| CA    | Consumo de Água                                                   |
| CBA   | Consumo de Bebidas Alcoólicas                                     |
| MCC   | Monitorização do Consumo Calorias Histórico de Obesidade Familiar | 

#### Informação sobre o dataset

A função ``info()`` apresenta a informação das colunas (atríbutos): Index da coluna, nome (se tiver, quantidade de registos *não nulos* e o tipo de dados de cada um deles.

In [ ]:
dados_analise.info()

#### Amostra do dataset

Para uma rápida visualização dos dados, temos as funções ``head(n)`` e ``tail(n)`` que nos dão *n* linhas, por defeito **5**, respetivamente do início ou fim do **dataset**.

In [ ]:
dados_analise.head()

In [ ]:
dados_analise.tail()

#### Descrição do dados do dateset

A função ``describe()`` analisa os dados do **dataset** e retorna um conjunto de estatísticas descritivas.

In [ ]:
dados_analise.describe(include="all")

### Identificar as classes dos atributos

Os atributos numéricos serão trabalhados mais adiante, quando fizermos a sua **normalização**. Para já vamos verificar as classes existentes para decidir qual a melhor estratégia para os preparar para a utilização nos vários métodos que vamos utilizar.

#### Genero

In [ ]:
titulo("Classes da coluna \033[1mGenero\033[0m")
for classe in dados_analise['Genero'].unique():
    texto(classe)

#### Historico_obesidade_familiar

In [ ]:
titulo("Classes da coluna \033[1mHistorico_obesidade_familiar\033[0m")
for classe in dados_analise['Historico_obesidade_familiar'].unique():
    texto(classe)

#### FCCAC

In [ ]:
titulo("Classes da coluna \033[1mFCCAC\033[0m")
for classe in dados_analise['FCCAC'].unique():
    texto(classe)

#### CCER

In [ ]:
titulo("Classes da coluna \033[1mCCER\033[0m")
for classe in dados_analise['CCER'].unique():
    texto(classe)

#### Fumador

In [ ]:
titulo("Classes da coluna \033[1mFumador\033[0m")
for classe in dados_analise['Fumador'].unique():
    texto(classe)

### MCC

In [ ]:
titulo("Classes da coluna \033[1mMCC\033[0m")
for classe in dados_analise['MCC'].unique():
    texto(classe)

#### CBA

In [ ]:
titulo("Classes da coluna \033[1mCBA\033[0m")
for classe in dados_analise['CBA'].unique():
    texto(classe)

#### TRANS

In [ ]:
titulo("Classes da coluna \033[1mTRANS\033[0m")
for classe in dados_analise['TRANS'].unique():
    texto(classe)

#### Label

In [ ]:
titulo("Classes da coluna \033[1mLabel\033[0m")
for classe in dados_analise['Label'].unique():
    texto(classe)

### 4.1.2



#### Derivar atributo (IMC)

Depois de criar o novo atributo, podemos usar qualquer uma das funções que foram utilizadas para analisar o **dataset**, neste caso tratando-se de valores numéricos consideramos que a que a mais útil é a ``describe()``.

Podemos também remover as colunas utilizadas para este cálculo uma vez que não vão ser necessárias.

Faz sentido alguma limpeza do **Dataset** neste momento, pois vai facilitar o nosso trabalho.

In [ ]:
dados_analise['IMC'] = dados_analise['Peso'] / (dados_analise['Altura'] * dados_analise['Altura'])
dados_analise.drop(columns=['Peso', 'Altura'], inplace=True)

#### Rever o **dataset** após as alterações

In [ ]:
dados_analise.describe(include="all")

Finalmente, usamos novamente a função ``info()`` para confirmar as *series* com que vamos trabalhar de seguida. 

In [ ]:
dados_analise.info()

### 4.1.3 Analisar os atributos do conjunto de dados mais significativos, usando gráficos, ...


#### Histogramas

Com estes gráficos podemos ver a distribuição dos valores por "segmentos" de valores.

In [ ]:
if not skip_graficos:
    sns.set(style="whitegrid")
    hist_columns_plot = ['FCV', 'NRP', 'CA', 'FAF', 'TUDE', 'IMC']

    for idx, etiqueta_serie in enumerate(hist_columns_plot):
        histograma(
            dados=dados_analise[etiqueta_serie],
            grupos=20,
            kde=True,
            size_x=10,
            size_y=4,
            titulo=f"Distribuição de {etiqueta_serie}",
            etiqueta_x=etiqueta_serie,
            etiqueta_y="Frequência",
            color="skyblue",
            edgecolor="black",
        )

#### Pie Charts

A função ``tarte()`` retorna a ``Fig`` e o ``Axes`` para se poder personalizar mais o gráfico.
É também possível passar parâmetros adicionais (*kwargs*), os quais serão usados na invocação de ``pie()``. 

In [ ]:
if not skip_graficos:
    tarte(
        dados=dados_analise["Historico_obesidade_familiar"].value_counts(),
        size_x=4,
        size_y=4,
        titulo="Histórico obesidade familiar",
        labels=dados_lidos["Historico_obesidade_familiar"].unique(),
    )

    tarte(
        dados=dados_analise["FCCAC"].value_counts(),
        size_x=5,
        size_y=5,
        titulo="Frequência de Consumo de Comida Altamente Calórica",
        labels=dados_lidos["FCCAC"].unique(),
    )

    tarte(
        dados=dados_analise["Fumador"].value_counts(),
        size_x=5,
        size_y=5,
        titulo="Fumador",
        labels=dados_lidos["Fumador"].unique(),
    )

    tarte(
        dados=dados_analise["MCC"].value_counts(),
        size_x=5,
        size_y=5,
        titulo="Monitorização do Consumo Calorias",
        labels=dados_lidos["MCC"].unique(),
    )

    tarte(
        dados=dados_analise["CBA"].value_counts(),
        size_x=5,
        size_y=5,
        titulo="Consumo de Bebidas Alcoólicas",
        labels=dados_lidos["CBA"].unique(),
    )

    tarte(
        dados=dados_analise["TRANS"].value_counts(),
        size_x=5,
        size_y=5,
        titulo="Transporte",
        labels=[undersplit(texto) for texto in dados_lidos["TRANS"].unique()],
    )

    tarte(
        dados=dados_analise["Label"].value_counts(),
        size_x=5,
        size_y=5,
        titulo="Classificação do peso",
        labels=[undersplit(texto) for texto in dados_lidos["Label"].unique()],
    )

#### visualização de outliers

In [ ]:
if not skip_graficos:
    boxplots_colunas = ["IMC", "Idade"]

    for coluna in boxplots_colunas:
        if (dados_analise[coluna].dtype != 'object'):
            dados_analise.boxplot(
                column=[coluna],
                by="Label",
                grid=False,
                figsize=(8, 6),
                return_type="dict",
                rot=90,
            )
            plt.title(coluna)

    plt.show()

In [ ]:
if not skip_graficos:
    sns.pairplot(data=dados_analise, x_vars=colunas_numericas, y_vars="IMC")

### 4.1.4

#### a) Identificação de NAN e "limpar" o dataset.

Verificar se existem valores nulos.

In [ ]:
dados_analise[dados_analise.isnull().any(axis=1)]

Verificar se existem valores NaN

In [ ]:
dados_analise[dados_analise.isna().any(axis=1)]

#### Notas sobre o pré-processamento dos dados

Não foram encontrados valores nulos, nem valor **NaN**, não sendo por isso necessário fazer tratamento adicionais ao dados neste contexto. 

Algum do trabalho de limpeza foi realizado do ponto *4.1.2*. 

TODO: Comentário sobre dados inconsistentes e outliers
TODO: Selecionar atributos (?)


#### Normalização

A normalização consiste em transformar o intervalo de valores de um atributo em valores compreendidos entre **0** e **1**, para eliminar os efeitos da escala utilizada.

Podemos utilizar o ``MinMaxScaler``. Apenas as colunas de valores numéricos devem ser normalizadas:

Idade, FCV, NRP, CA, FAF, TUDE, IMC
 

In [ ]:
scaler = MinMaxScaler()
dados_analise.loc[:, colunas_numericas] = scaler.fit_transform(dados_analise.loc[:, colunas_numericas])
dados_analise

#### Transformar classes em valores

Vamos fazer esta operação para as colunas em que apenas temos 2 classes:

* genero
* historico_obesidade_familiar
* FCCAC
* Fumador
* MCC

NOTA: No mesmo passo em que fazemos a conversão, mostramos os valores finais.

In [ ]:
for coluna in colunas_classes_binarias:
    dados_analise[coluna] = label_encoder.fit_transform(dados_analise[coluna].values)
    etiqueta_e_valor(coluna, dados_analise[coluna].unique())

Para as colunas de classes com mais de 2 valores, vamos precisar de 2 abordagens, em algumas das operaçoes que vamos realizar precisamos de codificar os dados em valores numéricos, mas numa única coluna, enquanto que para outros precisamos de criar colunas "dummy" unado a função ``get_dummies`` do **pandas**.

Começamos por fazer um duplicado dos dados e tratar cada conjunto de uma forma diferente.

In [ ]:
dados_com_dummies = dados_analise.copy()
dados_trabalho = dados_analise.copy()

#### Encoding das colunas com múltiplos valores

In [ ]:
for coluna in colunas_classes_multiplos:
    dados_trabalho[coluna] = label_encoder.fit_transform(dados_trabalho[coluna].values)
    etiqueta_e_valor(coluna, dados_trabalho[coluna].unique())

#### Gerar Dummies para as colunas de classes



CCER, CBA e TRANS

In [ ]:
dados_com_dummies = pd.get_dummies(dados_com_dummies, dtype=float)

O **DataFrame** *dados_com_dummies* vai ser preciso para o ponto *4.2*, vamos guardar num ficheiro, que será depois aberto no outro notebook.

In [ ]:
dados_com_dummies.to_csv(ficheiro_dummies)

### 4.1.5 Matriz de Correlação

Para esta matriz vamos utilizar os dados previamente guardados (**dados_trabalho**) antes da preparação dos atributos para utilização nas várias metodologias que se seguem.  

In [ ]:
fig_corr = plt.figure(figsize=(12, 8))
etiquetas_corr = [undersplit(texto) for texto in dados_trabalho.columns]

for coluna in dados_trabalho.columns:
    dados_trabalho[coluna] = label_encoder.fit_transform(dados_trabalho[coluna].values)

sns.heatmap(dados_trabalho.corr(method="spearman").abs(), cmap='coolwarm', annot=True, fmt='.2f', mask=False)

plt.xticks(ticks=[i + 0.5 for i in range(len(dados_trabalho.columns))], labels=etiquetas_corr, rotation=45, ha='right')
plt.yticks(ticks=[i + 0.5 for i in range(len(dados_trabalho.columns))], labels=etiquetas_corr)
plt.title('Matriz de Correlação')
plt.tight_layout()

# plt.show()

Por este diagrama podemos ver que não existe uma forte correlação entre nenhum dos atributos. 

### 4.1.6

#### Hold-out

**Hold-out** é o processo de dividir um conjunto de dados em 2 sub-conjuntos de forma aleatória, um de maior dimensão que será utilizado para treinar um algoritmo de Machine Learning e o outra para o testar.


| Predictor | Target  |
|-----------|---------|
| Idade     | IMC     |

In [ ]:
y = dados_trabalho.IMC
X = dados_trabalho[["Idade"]].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=100)

#### Apresentar informação sobre os datasets de treino e aprendizagem

In [ ]:
titulo("|\t\t\t\tDimensão\t\t\t|")
titulo("|   |\tTreino\t\t|\tTeste\t\t|")
texto(f"| X |\t{X_train.shape}\t|\t{X_test.shape}\t|")
texto(f"| y |\t{y_train.shape}\t\t|\t{y_test.shape}\t\t|")

#### Inicialização do algoritmo

In [ ]:
regr_lin = LinearRegression()

#### Treinar o algoritmo

Para o treino utilizamos a função ``fit()``.

In [ ]:
regr_lin.fit(X_train, y_train)

#### 4.1.6 a)

In [ ]:
etiqueta_e_valor("Equação da reta:", f" y = {regr_lin.intercept_:.3f} + {regr_lin.coef_[0]:.3f} x")

$$y = 0,355 + 0,381x$$

#### 4.1.6 b)

In [ ]:
sns.regplot(x="Idade", y="IMC", data=dados_trabalho, order=1, ci=None, scatter_kws={"color": "b", "s": 0.1})
plt.xlim(10, 65)
plt.ylim(ymin=10);

Aplicar o modelo sobre o conjunto de teste.

In [ ]:
y_pred = regr_lin.predict(X_test)

#### MAE

$$MAE = \sum_{i=1}^{D}|x_i-y_i|$$

In [ ]:
MAE = np.mean(abs(y_test - y_pred))
etiqueta_e_valor("MAE no conjunto de dados de teste:", f"{MAE:.3f}")

#### RMSE

$$RMSE = \sqrt{\frac{\sum_{i=0}^{N - 1} (y_i - \hat{y}_i)^2}{N}}$$

In [ ]:
RMSE = np.sqrt(np.mean((y_test - y_pred) ** 2))
etiqueta_e_valor("RMSE no conjunto de dados de teste:", f"{RMSE:.3f}")

#### Repetir o cálculo mas com outros previsores (preditors)

O preditor vai mudando, podemos ter uma lista com o nome das colunas e fazer um ciclo que realiza os cálculos e guardamos o resultado num dicionário.

TODO: Utilizam-se os atributos base ou os antes do tratamento de dados.   

In [ ]:
dados_slr = dados_trabalho.copy()

for coluna in dados_slr.columns:
    dados_slr[coluna] = label_encoder.fit_transform(dados_slr[coluna].values)


In [ ]:
# O target não vai mudar.
y = dados_trabalho.IMC

# O preditor vai mudando, podemos ter uma lista com o nome das colunas
# lista_preditores = ['Genero', 'Historico_obesidade_familiar', 'FCCAC', 'FCV', 'NRP', 'CCER', 'Fumador', 'CA', 'MCC', 'FAF', 'TUDE', 'CBA', 'TRANS']

lista_preditores = dados_trabalho.columns

# Resultados
resultados = {}

for preditor in lista_preditores:
    X_preditor = dados_trabalho[[preditor]].to_numpy()
    X_train_1, X_test_1, y_train, y_test = train_test_split(X_preditor, y, train_size=0.8, test_size=0.2,
                                                            random_state=100)
    line_regr = LinearRegression()
    modelo = line_regr.fit(X_train_1, y_train)

    y_pred_1 = line_regr.predict(X_test_1)

    resultados[preditor] = {
        # "X_train": X_train_1,
        # "X_test": X_test_1,
        # "y_train": y_train,
        # "y_test": y_test,
        "modelo": modelo,
        "y_pred": y_pred_1,
        "MAE": np.mean(abs(y_test - y_pred_1)),
        "RMSE": np.sqrt(np.mean((y_test - y_pred_1) ** 2))
    }


#### Tabela com os resultados

Uma vez que temos o pandas, podemos utilizar este para apresentar a informação em formato de tabela.


TODO: Agora é preciso analisar os resultados e talvez produzir algum gráfico ou análise estatistica.

In [ ]:
resultado = pd.DataFrame(resultados)

resultado.loc[["MAE", "RMSE"]]


### 4.1.7

#### a) Utilizar Regressão Linear Múltipla

| Target | Preditores |
|--------|------------|
| IMC    | ??         |

Ver TP5


Holdout



In [ ]:
y = dados_trabalho.IMC
X = dados_trabalho[lista_preditores].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                    random_state=100)

TODO: Imprimir a equação corretamente
TODO: Rever o código é cópia do TP5 

In [ ]:
## Initialize algorithm
mlr = LinearRegression()

## Fit the data
mlr.fit(X_train, y_train)

print("Eq. da reta: y=", mlr.intercept_, "+", mlr.coef_, "x")

In [ ]:
#Prediction of test set
y_pred_mlr = mlr.predict(X_test)
#Predicted values
#print("Prediction for test set: {}".format(y_pred_mlr))

In [ ]:
#Actual value and the predicted value
mlr_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': y_pred_mlr})
mlr_diff.head()

In [ ]:
#Model Evaluation
from sklearn import metrics

meanAbErr = metrics.mean_absolute_error(y_test, y_pred_mlr)
meanSqErr = metrics.mean_squared_error(y_test, y_pred_mlr)

rootMeanSqErr = np.sqrt(metrics.mean_squared_error(y_test, y_pred_mlr))

print('R squared: {:.2f}'.format(mlr.score(X, y) * 100))
print('Mean Absolute Error:', meanAbErr)
print('Mean Square Error:', meanSqErr)
print('Root Mean Square Error:', rootMeanSqErr)

#### b) DecisionTreeRegressor

Vamos usar os mesmos sets de treino e de teste.

TODO: Rever o código é cópia do TP5, ver TP7 tem uns testes que talvez sejam necessários.

In [ ]:
arvore_decisao = DecisionTreeRegressor(
    random_state=42,
    max_depth=6,
    min_samples_split=3
)  # Restrição de alguns parâmetros da árvore
modelo_regressao = arvore_decisao.fit(X_train, y_train)

y_pred = arvore_decisao.predict(X_train)
y_pred1 = arvore_decisao.predict(X_test)

In [ ]:
MAE1 = metrics.mean_absolute_error(y_train, y_pred)
MAE2 = metrics.mean_absolute_error(y_test, y_pred1)
MAE3 = np.mean(abs(y_test - y_pred1))  #efetuando os cálculos

print("MAE on training set: {:.3f}".format(MAE1))
print("MAE on test set: {:.3f}".format(MAE2))
print("MAE on test set: {:.3f}".format(MAE3))  #efetuando os cálculos

RMSE = np.sqrt(np.mean((y_test - y_pred1) ** 2))
print("RMSE: {:.4f}".format(RMSE))

Visualização da Regression Tree

TODO: QUe valores utilizar?

In [ ]:
tree.plot_tree(
    modelo_regressao,
    feature_names=list(dados_trabalho.columns),
    class_names=list(dados_trabalho['IMC']),
    filled=True,
    fontsize=6
);

#### Modelo de regressão

In [ ]:
print(export_text(modelo_regressao, show_weights=True))

#### c) MLPRegressor

TODO: Rever código é cópia do TP7

In [ ]:
Nhidden = 1  # (?)

nn = MLPRegressor(hidden_layer_sizes=Nhidden,
                  activation='tanh',
                  solver='lbfgs', max_iter=1000, learning_rate_init=0.001)

In [ ]:
nn.fit(X_train, y_train)

In [ ]:
# Make prediction
pred = nn.predict(X_test)
#
# Calculate accuracy and error metrics
#
test_set_rsquared = nn.score(X_test, y_test)
test_set_rmse = np.sqrt(mean_squared_error(y_test, pred))
#
# Print R_squared and RMSE value
#
print('R_squared value: ', test_set_rsquared)
print('RMSE: ', test_set_rmse)

In [ ]:
print("weights between input and first hidden layer:")
print(nn.coefs_[0])
print("\nweights between first hidden and second hidden layer:")
print(nn.coefs_[1])

### 4.1.8

TODO Realizar os cálculos do MAE e RMSE do 4.1.7 colunas num DataFrame e imprimir

In [ ]:
comparar_417 = pd.DataFrame()
comparar_417

### 4.1.9

TODO: Estudo estatistico (ttest_ind) ? para os 2 modelos que apresentem melhores resultados.